<u><b><h1 align="center" style = "color:Red; background-color:yellow;" > Data Science Regression Project: Predicting Home Prices in Banglore </h1></b></u>

<u><b><h3 style = "color:Blue"> Loading Data</h3></b></u>

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)

In [ ]:
df1 = pd.read_csv("Bengaluru_House_Data.csv")
print(df1.shape)
df1.head()

In [ ]:
df1["area_type"].unique()

In [ ]:
df1['area_type'].value_counts()

<b> Dropping features that are not required to build our model </b>

In [ ]:
df2 = df1.drop(["area_type", "availability", "society", "balcony"], axis = "columns")
df2.head()

<u><b><h3 style = "color:Blue;">Data Cleaning</h3></b></u>

In [ ]:

df2.isnull().sum()

In [ ]:
df3 = df2.dropna()  # Since the NaN values in each column were very few as compared to total rows in the dataset ; hence we can easily drop those rows.
df3.shape

In [ ]:
df3["size"].unique()

Now it's evident that the BHK and Bedroom are one and all the same thing ; but since the string value is different hence they are treated different ; for that purpose we can make a column named "Size_in_BHK".
For that make a function that retrieve the Integer value from the size column.

<b> Add new feature(integer) for bhk (Bedrooms Hall Kitchen) </b>

In [ ]:
df3['Size_in_BHK'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))
df3.head()

In [ ]:
df3.drop(["size"], axis = "columns", inplace=True)

In [ ]:
df3["total_sqft"].unique()

Check that the values are not one having single number  ; few datas of <b>total_sqft</b> have ranges too ; like 1133-1384. 
We need to remove these. 
<br>For this we can check if the total_sqft value is float or not.

In [ ]:
def is_float(x):
    try:
        float(x)   # If float value exists then it'll return ; as the range can't be converted to float value.
    except:
        return False
    return True

In [ ]:
df3[df3["total_sqft"].apply(is_float)]     # Checking for valid numerical value of "total_sqft"

In [ ]:
df3[~df3["total_sqft"].apply(is_float)].head(15) # Negation used here ; will throw dataset where the "total_sqft" value is range type

Above shows that total_sqft can be a range (e.g. 2100-2850). For such case we can just take average of min and max value in the range. There are other cases such as 34.46Sq. Meter which one can convert to square ft using unit conversion.We will just drop such corner cases to keep things simple.

In [ ]:
# Function to convert ranges to number (We will take average of upper limit and lower limit)

def range_sqft_to_num (x):
    tokens = x.split("-")
    if(len(tokens)==2):
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [76]:
df4 = df3.copy()
df4.total_sqft = df4["total_sqft"].apply(range_sqft_to_num)

# Now we will drop NaN values of total_sqft column which we had converted to NaN by changing the extra values like 34.46 sq. meter etc. to NaN 

df4.total_sqft = df4.dropna(subset = ["total_sqft"], inplace=True)   # we could also use this command :  df4 = df4[df4.total_sqft.notnull()]
df4.shape

(13200, 5)

We have reduced the no. of rows to 13200 which were 13320 in the original dataset. 